In [1]:
qi("AmpelContext", "AmpelDB", "AlertProcessor", "TarAlertLoader")

In [2]:
ctx = AmpelContext.load("ampel_conf.json", tier=2)
dict.__setitem__(ctx.config._config['db'], 'prefix', 'AmpelTest')
AmpelDB.delete_ampel_databases(ctx.config, "AmpelTest")

In [3]:
ap = AlertProcessor(
    context = ctx,
    publish_stats = ["mongo"],
    supplier = {
        "unit": "FilteringAlertSupplier",
        "config": {
            "supplier": {"unit": "ZiAlertSupplier"},
            "match_ids": [770239962715015024]
        }
    },
    logger_profile = "quiet",
    directives = [
        {
            "channel": "VAL_TEST1", 
            "filter": {"unit": "RandFilter", "config": {"passing_rate": 1.}},
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [{'unit': 'T2SNCosmo', 'config': 55880712071630791}]
                        }
                    }
                ]
            }
        }
    ]
)


2020-06-02 18:01:54 AmpelDB:160 INFO
 Creating AmpelTest_data -> stock
   Creating index: {'index': [('_id', 1), ('channel', 1)], 'args': {'unique': True}}

2020-06-02 18:01:54 CORE FilterBlock:78 INFO
 Loading filter: RandFilter

2020-06-02 18:01:54 UNIT RandFilter:22 INFO
 RandFilter initialized with passing rate 1.0
 AlertProcessor setup completed


In [4]:
ap.set_loader(
    TarAlertLoader('/Users/hu/Documents/ZTF/ztf_public_20190210.tar.gz')
)
ap.run()

 Creating AmpelTest_ext -> counter
 Creating AmpelTest_var -> logs
   Creating index: {'index': [('run', 1)]}
   Creating index: {'index': [('stock', 1)], 'args': {'sparse': True}}
   Creating index: {'index': [('channel', 1)], 'args': {'sparse': True}}
 Creating AmpelTest_var -> events
 Creating AmpelTest_data -> t2
   Creating index: {'index': [('stock', 1), ('channel', 1)]}
   Creating index: {'index': [('status', 1)]}
 Creating AmpelTest_data -> t0
   Creating index: {'index': [('stock', 1)]}
 Creating AmpelTest_data -> t1
   Creating index: {'index': [('stock', 1)]}

2020-06-02 18:01:56 CORE AlertProcessor:332 SHOUT
 Processing alerts (run: 1)

2020-06-02 18:02:45 TarAlertLoader:71 INFO
 Reached end of tar file /Users/hu/Documents/ZTF/ztf_public_20190210.tar.gz

2020-06-02 18:02:45 CORE AlertProcessor:542 SHOUT
 Processing completed (time required: 51s)


1